# Planner calculations

In [4]:
# Import necessary libraries
import xml.etree.ElementTree as ET
import pandas as pd

# Function to parse tasks with WBS
def parse_tasks_no_duplicates(tasks, parent_wbs="", top_level_counter=[1], processed_ids=set()):
    wbs_list = []
    local_counter = 1

    for task in tasks:
        task_id = task.get("id", "")
        if task_id in processed_ids:  # Skip duplicates
            continue
        processed_ids.add(task_id)

        name = task.get("name", "")
        duration = task.get("duration", "0")
        start = task.get("start", "")[:8]  # Trimming to YYYYMMDD format
        end = task.get("end", "")[:8]     # Trimming to YYYYMMDD format

        # Generate WBS
        if parent_wbs:
            current_wbs = f"{parent_wbs}.{local_counter}"
            local_counter += 1
        else:  # Top-level task
            current_wbs = str(top_level_counter[0])
            top_level_counter[0] += 1

        # Append task data
        wbs_list.append({
            "WBS": current_wbs,
            "ID": task_id,
            "Name": name,
            "Duration (Workdays)": int(duration) / (8 * 3600),
            "Start": start,
            "End": end,
            "Milestone": int(duration) == 0
        })

        # Recurse into child tasks
        child_tasks = task.findall("./task")
        if child_tasks:
            wbs_list.extend(parse_tasks_no_duplicates(
                child_tasks, parent_wbs=current_wbs, top_level_counter=top_level_counter, processed_ids=processed_ids
            ))

    return wbs_list

# Load XML file
xml_file_path = "CZPAD.planner"  # Replace with the path to your file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Parse tasks and create a DataFrame
root_tasks = root.findall(".//task")
parsed_tasks = parse_tasks_no_duplicates(root_tasks)
tasks_df = pd.DataFrame(parsed_tasks)

# Convert Start and End to datetime
tasks_df["Start"] = pd.to_datetime(tasks_df["Start"], format="%Y%m%d")
tasks_df["End"] = pd.to_datetime(tasks_df["End"], format="%Y%m%d")

# Display the resulting DataFrame (optional)
tasks_df


,WBS,ID,Name,Duration (Workdays),Start,End,Milestone
0,1,1,KO,0.000000,2025-04-01,2025-04-01,True
1,2,2,Analysis and Design,80.000000,2025-04-01,2025-07-21,False
2,2.1,3,Review of current FM (TRL9),20.000000,2025-04-01,2025-04-28,False
3,2.2,4,Mechanical and thermal redesign,30.000000,2025-04-28,2025-06-09,False
4,2.3,5,Electrical redesign,30.000000,2025-04-28,2025-06-09,False
5,2.4,6,Wearing design,30.000000,2025-06-09,2025-07-21,False
6,2.5,7,SSR (System Requirement Review),0.000000,2025-06-09,2025-06-09,True
7,3,8,Manufacturing and Testing EM,90.000000,2025-06-09,2025-10-13,False
8,3.1,9,EM manufacturing (TRL7),30.000000,2025-06-09,2025-07-21,False
9,3.2,10,Battery module EQM manufacturing (TRL6),30.000000,2025-06-09,2025-07-21,False


In [7]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
xml_file_path = "CZPAD.planner"
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Parse tasks from XML
def parse_tasks_with_wbs(tasks, parent_wbs="", top_level_counter=[1], processed_ids=set()):
    wbs_list = []
    local_counter = 1

    for task in tasks:
        task_id = task.get("id", "")
        if task_id in processed_ids:  # Skip duplicates
            continue
        processed_ids.add(task_id)

        name = task.get("name", "")
        duration = task.get("duration", "0")
        start = task.get("start", "")[:8]  # Trimming to YYYYMMDD format
        end = task.get("end", "")[:8]     # Trimming to YYYYMMDD format

        # Generate WBS
        if parent_wbs:
            current_wbs = f"{parent_wbs}.{local_counter}"
            local_counter += 1
        else:  # Top-level task
            current_wbs = str(top_level_counter[0])
            top_level_counter[0] += 1

        # Append task data
        wbs_list.append({
            "task-id": task_id,
            "name": name,
            "duration": duration,
            "start": start,
            "end": end,
            "WBS": current_wbs
        })

        # Recurse into child tasks
        child_tasks = task.findall("./task")
        if child_tasks:
            wbs_list.extend(parse_tasks_with_wbs(
                child_tasks, parent_wbs=current_wbs, top_level_counter=top_level_counter, processed_ids=processed_ids
            ))

    return wbs_list

# Parse tasks
root_tasks = root.findall(".//task")
tasks_data = parse_tasks_with_wbs(root_tasks)
tasks_with_wbs_df = pd.DataFrame(tasks_data)

# Parse allocations and calculate workdays
def parse_allocations_with_durations(root, tasks_df):
    task_duration_map = tasks_df.set_index("task-id")["duration"].astype(float) / (8 * 3600)  # Convert seconds to workdays
    
    allocations = []
    for allocation in root.findall(".//allocation"):
        task_id = allocation.get("task-id")
        resource_id = allocation.get("resource-id")
        units = float(allocation.get("units", "0")) / 100.0  # Convert percentage to fraction
        
        if task_id in task_duration_map.index:
            allocations.append({
                "task-id": task_id,
                "resource-id": resource_id,
                "units": units,
                "workdays": units * task_duration_map[task_id]  # Calculate resource workdays
            })

    return pd.DataFrame(allocations)

# Parse allocations with durations
allocations_with_workdays_df = parse_allocations_with_durations(root, tasks_with_wbs_df)

# Pivot the table to have resource-ids as columns
allocations_pivot = allocations_with_workdays_df.pivot_table(
    index="task-id", 
    columns="resource-id", 
    values="workdays", 
    fill_value=0
)

# Rename columns based on short-name
resources_df = pd.DataFrame([
    {"resource-id": res.get("id"), "short-name": res.get("short-name")}
    for res in root.findall(".//resource")
])
resource_map = resources_df.set_index("resource-id")["short-name"].to_dict()
allocations_pivot.columns = [resource_map.get(col, f"Resource-{col}") for col in allocations_pivot.columns]

# Add WBS to the table
allocations_pivot["WBS"] = tasks_with_wbs_df.set_index("task-id")["WBS"]

# Reorder columns to place WBS at the beginning
columns = ["WBS"] + [col for col in allocations_pivot.columns if col != "WBS"]
allocations_df = allocations_pivot[columns]

# Ensure task-id is numeric and sort
allocations_df = allocations_df.reset_index()
allocations_df["task-id"] = allocations_df["task-id"].astype(int)
allocations_df = allocations_df.sort_values(by="task-id").set_index("task-id")

# The resulting dataframe is named `allocations_df`
allocations_df


,WBS,EE,PROJ,UST,PR,ME,SCI,FW,SUP
task-id,,,,,,,,,
3,2.1,10,0,0,0,10,10,0,0
4,2.2,0,0,0,0,30,0,0,0
5,2.3,30,0,0,0,0,0,0,0
6,2.4,0,0,0,0,30,0,0,0
9,3.1,0,0,30,0,0,0,0,0
10,3.2,0,0,30,0,0,0,0,0
11,3.3,0,0,0,0,0,0,60,0
12,3.4,30,0,0,0,30,0,0,0
15,4.1,0,0,30,0,0,0,0,0


In [11]:
# Extract the first digit of the WBS and create a grouping column
allocations_df["WBS_Group"] = allocations_df["WBS"].str.split('.').str[0]

# Group by the first digit of the WBS and sum rows for each group
grouped_allocations = allocations_df.groupby("WBS_Group").sum(numeric_only=True)

# Drop the grouping column if not needed in the final output
grouped_allocations = grouped_allocations.drop(columns=["WBS_Group"], errors="ignore")

# Display the grouped DataFrame
print(grouped_allocations)


           EE  PROJ  UST   PR  ME  SCI  FW  SUP
WBS_Group                                      
2          40     0    0    0  70   10   0    0
3          30     0   60    0  30    0  60    0
4          60     0   60    0  74   30   0   60
5          40     0   30    0  40   10  60   30
6          15     0    0    0  15  160   0   44
7           0     0    0  281   0   60   0    0
8           0   280    0    0   0    0   0    0


In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "team_structure.csv"
team = pd.read_csv(file_path)

# Display the DataFrame for verification
print(team)


    team    team-leader           team-member  FTE
0     EE   Jakub Kakona          Jakub Kakona  1.0
1     EE   Jakub Kakona  Electrical Engineer2  1.0
2     ME   Roman Dvorak          Roman Dvorak  1.0
3     ME   Roman Dvorak  Mechanical Engineer2  0.5
4    SCI    Ondrej Ploc           Ondrej Ploc  0.5
5    SCI    Ondrej Ploc         Iva Ambrozova  0.5
6    SCI    Ondrej Ploc         Martin Kakona  0.8
7    SCI    Ondrej Ploc           Jakub Slegl  0.5
8     FW   Vit Hanousek          Vit Hanousek  1.0
9     FW   Vit Hanousek          Programator2  0.5
10   SUP  Iva Ambrozova           Technician1  0.5
11   SUP  Iva Ambrozova           Technician2  0.5
12  PROJ  Iva Ambrozova         Iva Ambrozova  0.3
13  PROJ  Iva Ambrozova                Roxana  0.5
14   UST     Fluktuacia                   NaN  NaN
15    PR  Iva Ambrozova           Kosmonautix  0.2


In [10]:
# Compute total FTE for each team
team_fte = team.groupby("team")["FTE"].sum()

# Initialize a dictionary to store total workdays for each team
team_total_workdays = {}

# Iterate over each team in the allocations_df
for team_name in allocations_df.columns:
    if team_name != "WBS":  # Skip WBS column
        # Sum the workdays for the team
        team_workdays = allocations_df[team_name].sum()
        
        # Multiply by the total FTE for the team
        team_total_fte = team_fte.get(team_name, 0)  # Default FTE is 0 if team is not in `team`
        team_total_workdays[team_name] = team_workdays * team_total_fte

# Convert the result to a DataFrame
team_workdays_df = pd.DataFrame.from_dict(
    team_total_workdays, orient="index", columns=["Total Workdays"]
)
team_workdays_df.index.name = "Team"

# Display the resulting DataFrame
print(team_workdays_df)


      Total Workdays
Team                
EE             370.0
PROJ           224.0
UST              0.0
PR              56.2
ME             343.5
SCI            621.0
FW             180.0
SUP            134.0


In [13]:
# Compute total FTE for each team
team_fte = team.groupby("team")["FTE"].sum()

# Initialize a new DataFrame to store workdays per team member and WBS_Group
workdays_per_member = []

# Iterate over each WBS_Group in grouped_allocations
for wbs_group, row in grouped_allocations.iterrows():
    # Iterate over each team
    for team_name, total_workdays in row.items():
        if team_name in team_fte and total_workdays > 0:
            # Distribute workdays among team members based on their FTE
            team_members = team[team["team"] == team_name]
            for _, member in team_members.iterrows():
                member_workdays = total_workdays * (member["FTE"] / team_fte[team_name])
                workdays_per_member.append({
                    "WBS_Group": wbs_group,
                    "Team": team_name,
                    "Team Member": member["team-member"],
                    "Workdays": member_workdays
                })

# Convert to a DataFrame
workdays_per_member_df = pd.DataFrame(workdays_per_member)

# Display the resulting DataFrame
workdays_per_member_df

,WBS_Group,Team,Team Member,Workdays
0,2,EE,Jakub Kakona,20.000000
1,2,EE,Electrical Engineer2,20.000000
2,2,ME,Roman Dvorak,46.666667
3,2,ME,Mechanical Engineer2,23.333333
4,2,SCI,Ondrej Ploc,2.173913
5,2,SCI,Iva Ambrozova,2.173913
6,2,SCI,Martin Kakona,3.478261
7,2,SCI,Jakub Slegl,2.173913
8,3,EE,Jakub Kakona,15.000000
9,3,EE,Electrical Engineer2,15.000000


In [14]:
workdays_per_member_df.to_csv('workdays.csv')